https://www.kaggle.com/code/luanafelbarros/detecting-hate-speech-and-offensive-language

This website and the code on it were used as a reference for our work.

In [196]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import json
import os
import time
import pickle

df = pd.read_csv('labeled_data.csv')

In [197]:
df = df[['class', 'tweet']]

In [198]:

for i in range(24783):
    tweet = df['tweet'][i]
    review = re.sub(r"@[A-Za-z0-9_]+", " ", tweet)
    review = re.sub('RT', ' ', review)
    review = re.sub(r"https?://[A-Za-z0-9./]+", " ", review)
    review = re.sub(r"https?", " ", review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = ' '.join(review)
    df['tweet'][i] = review


C:\Users\15404\AppData\Local\Temp/ipykernel_20824/3878713545.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tweet'][i] = review


In [199]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   class   24783 non-null  int64 
 1   tweet   24783 non-null  object
dtypes: int64(1), object(1)
memory usage: 387.4+ KB


In [200]:
train_x = df['tweet'][0:22304]
train_y = df['class'][0:22304]
test_x = df['tweet'][22305:24781]
test_y = df['class'][22305:24781]

In [201]:
def tokenize(xtrain, xtest):
    tok = TweetTokenizer()
    vec = CountVectorizer(analyzer="word", tokenizer=tok.tokenize, max_features=1010)
    xtrain = vec.fit_transform(xtrain).toarray()
    xtest = vec.transform(xtest).toarray()
    return xtrain, xtest

train_x, test_x = tokenize(train_x, test_x)
    

In [202]:
from sklearn.svm import SVC, LinearSVC

svm = SVC(kernel='linear', random_state=42)

st = time.time()
svm.fit(train_x, train_y)
et1 = time.time()

print('[Training Time]: {:.1f} seconds'.format(et1 - st))

[Training Time]: 114.8 seconds


In [203]:
def model_accuracy(model, x, true_y):
    pred = model.predict(x)
    return np.sum(pred == true_y) / true_y.shape[0]

def evaluate_model(name, model, train_x, train_y, val_x, val_y):
    train_acc = model_accuracy(model, train_x, train_y)
    val_acc = model_accuracy(model, val_x, val_y)
    print('[{}] Training accuracy: {:.2f}%, Validation accuracy: {:.2f}%'.format(name, train_acc*100, val_acc*100))
    
evaluate_model('SVM', svm, train_x, train_y, test_x, test_y)

[SVM] Training accuracy: 92.14%, Validation accuracy: 91.52%


In [204]:
pickle.dump(svm, open('model.sav', 'wb'))